In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import http

drive.mount('/content/drive')

# 시가총액 상위 300개 기업리스트 (미래에셋 대우 API 미확인으로, 2021.09.12 기준 시총 상위 300개 기업 사용)
lists = pd.read_csv("/content/drive/MyDrive/미래에셋/kospi/KOSPI_시가총액_상위500.csv")
lists_1 = lists.loc[0:299]
lists_2 = lists_1[['종목명']]


# ESG 키워드
ESG = {
'E_keyword' : ['친환경', '탄소', '에너지'],
'S_keyword' : ['캠페인', '교육', '사회'],
'G_keyword' : ['위원회', '이사', '지배구조']}
ESG_keyword = pd.DataFrame.from_dict(ESG)

print(ESG_keyword)


Mounted at /content/drive
  E_keyword S_keyword G_keyword
0       친환경       캠페인       위원회
1        탄소        교육        이사
2       에너지        사회      지배구조


In [ ]:
# 지난 N개월 간의 종목명 + E 관련 뉴스기사 본문 추출
from multiprocessing import Pool
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
import urllib.request
from urllib.parse import quote
import pandas as pd


# 크롤링 메인 함수
def get_news(stock, keyword, query, sort, lastpage, period):

  stockName = stock
  ESGKeyword = keyword

  # 크롤링 결과 저장 데이터프레임
  news_df = pd.DataFrame(columns=("stockName", "ESGKeyword", "Title", "Article")) 

  idx = 0


  for month in range (0, period): # 해당 검색어(query)로 N(=period)개월치 기사 추출

    s_date = datetime.date.today() + timedelta(days = -30 * (month+1))
    e_date = datetime.date.today() + timedelta(days = -30 * month)

    pageNum = 1

    for i in range (1, int(lastpage)*10, 10):
      # print(f"/{pageNum}페이지입니다/")

      url_query = quote(query)
      url = f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={url_query}&sort=0&photo=0&field=0&pd=6&ds={s_date}&de={e_date}&nso=so:r,p:{period}m,a:all&start={i}"

      search_url = urllib.request.urlopen(url).read()
      soup = BeautifulSoup(search_url, 'html.parser')
      links = soup.find_all('div', {'class':'info_group'})
  

      # 하나의 검색 페이지 안에서의 반복
      for link in links:

        # '네이버 뉴스' 형식으로 작성된 기사만을 추출 (href 2번)
        news_urls = link.find_all('a')

        if (len(news_urls) == 1): # '네이버 뉴스' 형식 X
          continue

        else:  # '네이버 뉴스' 형식 O
          
          for news_url_test in news_urls:
            news_url = news_url_test.get('href')

          try:
            news_link = urllib.request.urlopen(news_url).read()
          except (http.client.IncompleteRead) as e:
            news_link = e.partial
          
          news_html = BeautifulSoup(news_link, 'html.parser')

          try: 
            title = news_html.find('h3', {'id': 'articleTitle'}).get_text()
            article = news_html.find('div', {'id': 'articleBodyContents'}).get_text()
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
            article = article.replace("function _flash_removeCallback() {}", "")
            article = article.replace("\n","")
            article = article.replace("\t","")

            news_df.loc[idx] = [stockName, ESGKeyword, title, article]
            idx += 1

          except: 
            pass

          # print("#", end="")


      pageNum += 1


  return news_df




In [ ]:
# E(환경)부문


# CONCAT 목적의 초기 데이터프레임 형성
default = {
'stockName' : ['Test'],
'ESGKeyword' : ['Test'],
'Title' : ['Test'],
'Article' : ['Test']}
df_default = pd.DataFrame.from_dict(default) 


# 파라미터 입력 후 크롤링
for i in range(100): #시총 상위 100개 기업

  query = lists_2['종목명'].values[i] + "+" + ESG_keyword['E_keyword'].values[0]
  sort = "0" # 관련도 순 기사
  lastpage = 100 # 100 페이지까지 크롤링 (최대 400페이지까지 가능하기는 하나, 파일 크기 과부하 방지)
  period = 6 # 6개월간의 기사

  # print(lists_2['종목명'].values[i])
  # print(ESG_keyword['E_keyword'].values[0])
  # print(i)
  
  df = get_news(lists_2['종목명'].values[i], ESG_keyword['E_keyword'].values[0], query, sort, lastpage, period)
  df_default = pd.concat([df_default, df], axis = 0)



In [ ]:
df_default.to_csv('E부문_상위100_친환경.csv', header = True, index = False)